In [4]:
system_message = """
    You are designing an encounter for Dungeons and Dragons 5th Edition.  You have the following information:
    Input Parameters:

    Player Information: Number of players and their respective levels.
    Desired Difficulty Level: Easy, Medium, Hard, or Deadly.
    Narrative and Setting Context: Brief description of the current narrative and setting in which the encounter will take place.
    
    As the encounter designer you should consider the following:

    Select Appropriate Creatures: Choose creatures with Challenge Ratings (CR) that match the desired difficulty level, ensuring that the total encounter difficulty is appropriate for the players' strength.
    Encounter Design Principles:
    Balance: Ensure that the encounter matches the desired difficulty level.
    Narrative Integration: Design the encounter to fit seamlessly into the ongoing story and setting.
    Variety: Introduce different types of challenges (combat, environmental, social) to keep the encounter interesting.
    Output of encounter details:
    Provide a list of creatures or adversaries, including their CRs.
    Suggest terrain and environmental features that enhance the encounter.
    Offer potential strategies or tactics that the creatures might use.
    Include any special conditions or objectives relevant to the encounter.
"""


Encounter Design:
For a medium difficulty encounter for three level 1 players in a city setting, we can focus on a combat encounter against a group of bandits who are causing trouble in the area. To add variety and make the encounter more interesting, we can also incorporate some social and environmental challenges.

Narrative and Setting Context:
The players are in a bustling city that has recently been plagued by a group of bandits. The bandits have been robbing travelers and causing fear among the citizens. The local authorities have put a bounty on the bandits' heads, and the players have decided to take on the task of bringing them to justice.

Encounter Details:
1. Creatures/Adversaries:
   - Bandit Captain (CR 2)
   - Bandit (CR 1/8) x3

2. Terrain and Environmental Features:
   - The encounter can take place in an alleyway in the city, with various crates and barrels providing cover.
   - There could be some narrow rooftops nearby, accessible through a series of skill checks, a

1. Include/generate enemy stat blocks
2. Wire it up - enemies should be initiatalized as agents.enemy and the DM should be initialized with all of the relevant characters sheets
3. Make sure CR is appropriate for the characters and requested difficulty
4. Make sure enemies and environment are thematically appropriate
5. Benchmark could implement CR calculator to evaluate options


In [9]:
from pydantic import BaseModel, Field
from typing import List

class Action(BaseModel):
    """A special move or tactic an enemy may take"""
    name: str = Field("Name of the action")
    description: str = Field("Description of what the action does and how it works")

class StatBlock(BaseModel):
    """Enemy stats definition, should include all innate abilities, spells, skill modifiers, armor class, and initiative"""
    armor_class: int = Field("The enemy's Armor Class")
    hit_point: int = Field("The enemy's Hit Points")
    speed: str = Field("The enemy's base movement speed")

    strength: str = Field("The enemy's strength (STR), should be of the form [[Value]] ([[Modifier]])")
    dexterity: str = Field("The enemy's dexterity (DEX), should be of the form [[Value]] ([[Modifier]])")
    constitution: str = Field("The enemy's constitution (CON), should be of the form [[Value]] ([[Modifier]])")
    intelligence: str = Field("The enemy's intelligence (INT), should be of the form [[Value]] ([[Modifier]])")
    wisdom: str = Field("The enemy's wisdom (WIS), should be of the form [[Value]] ([[Modifier]])")
    charisma: str = Field("The enemy's charisma (CHA), should be of the form [[Value]] ([[Modifier]])")

class Enemy(BaseModel):
    """Defines an enemy including all stats, conditions, and modifiers"""
    name: str = Field("The name of the creature or humanoid enemy")
    stats: StatBlock = Field("""The stat block for the enemy""")
    cr: str = Field("The enemy's Challenge Rating")
    actions: List[Action] = Field("A list of actions that the enemy may take")
    other_info: str = Field("Any other information relevant to how the enemy appears, acts, moves, or behaves")

class Encounter(BaseModel):
    """Defines an encounter including any enemies, terrain, setting, etc."""
    enemies: List[Enemy] = Field("List of enemies included in the encounter")

In [10]:
import simpleaichat

formatted_question = "Design an encounter for three level 5 players that is medium difficulty, and is set in a desert."


ai = simpleaichat.AIChat(console=False,
            save_messages=False,
            model="gpt-3.5-turbo",
            params={"temperature": 0.0},
            system=system_message
            )

response_structured = ai(
    formatted_question,
    output_schema=Encounter
)

print(response_structured)


{'enemies': [{'name': 'Sand Elemental', 'stats': {'armor_class': 15, 'hit_point': 90, 'speed': '30 ft., burrow 30 ft.', 'strength': '18 (+4)', 'dexterity': '14 (+2)', 'constitution': '16 (+3)', 'intelligence': '6 (-2)', 'wisdom': '10 (+0)', 'charisma': '6 (-2)'}, 'cr': '5', 'actions': [{'name': 'Slam', 'description': 'Melee Weapon Attack: +7 to hit, reach 5 ft., one target. Hit: 14 (2d8 + 5) bludgeoning damage.'}, {'name': 'Whirlwind', 'description': "Each creature in the elemental's space must make a DC 13 Strength saving throw. On a failure, a target takes 15 (3d8 + 2) bludgeoning damage and is flung up 20 feet away from the elemental in a random direction and knocked prone. If a thrown target strikes an object, such as a wall or floor, the target takes 3 (1d6) bludgeoning damage for every 10 feet it was thrown. If the target is thrown at another creature, that creature must succeed on a DC 13 Dexterity saving throw or take the same damage and be knocked prone."}], 'other_info': 'The

In [11]:
import json
print(json.dumps(response_structured["enemies"][0], indent=2))

    

{
  "name": "Sand Elemental",
  "stats": {
    "armor_class": 15,
    "hit_point": 90,
    "speed": "30 ft., burrow 30 ft.",
    "strength": "18 (+4)",
    "dexterity": "14 (+2)",
    "constitution": "16 (+3)",
    "intelligence": "6 (-2)",
    "wisdom": "10 (+0)",
    "charisma": "6 (-2)"
  },
  "cr": "5",
  "actions": [
    {
      "name": "Slam",
      "description": "Melee Weapon Attack: +7 to hit, reach 5 ft., one target. Hit: 14 (2d8 + 5) bludgeoning damage."
    },
    {
      "name": "Whirlwind",
      "description": "Each creature in the elemental's space must make a DC 13 Strength saving throw. On a failure, a target takes 15 (3d8 + 2) bludgeoning damage and is flung up 20 feet away from the elemental in a random direction and knocked prone. If a thrown target strikes an object, such as a wall or floor, the target takes 3 (1d6) bludgeoning damage for every 10 feet it was thrown. If the target is thrown at another creature, that creature must succeed on a DC 13 Dexterity savin